In [1]:
import pandas as pd
import math

In [2]:
data = pd.read_csv("eecs-445.csv")
data

,Unnamed: 0,Section,Instruction Mode,Class No,Enroll Stat,Open Seats,Wait List,Hour
0,3546,001 (LEC),In Person,26932,Open,120,-1,2021-11-15 06:00:00
1,3546,002 (LEC),In Person,15919,Open,4,10,2021-12-09 19:00:00
2,3547,002 (LEC),In Person,24565,Open,120,-1,2021-11-15 06:00:00
3,3547,010 (DIS),In Person,15922,Closed,0,3,2021-12-09 19:00:00
4,3548,011 (DIS),In Person,22551,Open,40,-1,2021-11-15 06:00:00
...,...,...,...,...,...,...,...,...
4123,5328,014 (DIS),In Person,26935,Closed,0,13,2021-12-09 19:00:00
4124,5329,015 (DIS),In Person,18524,Open,25,-1,2021-11-15 06:00:00
4125,5329,015 (DIS),In Person,26936,Closed,0,13,2021-12-09 19:00:00
4126,5330,016 (DIS),In Person,18504,Open,25,-1,2021-11-15 06:00:00


In [3]:
sortedData = data.sort_values(by=['Hour'], ascending=False)
data[data['Section']=='001 (LEC)'].sort_values(by=['Hour'], ascending=False)["Hour"]

4113    2021-12-09 19:00:00
15      2021-12-09 19:00:00
11      2021-12-09 19:00:00
4104    2021-12-09 16:00:00
4096    2021-12-09 15:00:00
               ...         
40      2021-11-15 10:00:00
32      2021-11-15 09:00:00
24      2021-11-15 08:00:00
16      2021-11-15 07:00:00
0       2021-11-15 06:00:00
Name: Hour, Length: 516, dtype: object

In [4]:
sortedData['Open Seats'][0] - sortedData['Open Seats'][len(data) - 0 - 1]
print(type(sortedData["Open Seats"]))

<class 'pandas.core.series.Series'>


In [12]:
daysList = []
for time in data['Hour']:
    daysList.append(time[0:10])
days = pd.Series(daysList).sort_values(ascending=False).unique()
days


array(['2021-12-09', '2021-12-08', '2021-12-07', '2021-12-06',
       '2021-12-05', '2021-12-04', '2021-12-03', '2021-12-02',
       '2021-12-01', '2021-11-30', '2021-11-29', '2021-11-28',
       '2021-11-27', '2021-11-26', '2021-11-25', '2021-11-24',
       '2021-11-23', '2021-11-22', '2021-11-21', '2021-11-20',
       '2021-11-19', '2021-11-18', '2021-11-17', '2021-11-16',
       '2021-11-15'], dtype=object)

In [13]:
# rate calculations
# I maybe should find the number of days each section has been open and divide by that
    # the final rate here is impacted by this (it seemed to close 2021-11-15 06:00:00 ?)
    # idk I assume this would primarily be the situation for sections and I would assume 
    # the opening days won't line up in any reasonable way
# random thought rn but like do we intend to leave this up for incoming freshman 
    # and so like throughout the summer ?
sections = data['Section'].unique()
rates = []
for className in sections:
    classData = data[data['Section']==className]
    classData = classData.sort_values(by=['Hour'], ascending=False).reset_index(drop=True)
    classData["Open Seats"][len(classData) - 1]
    rates.append([className, (classData["Open Seats"][len(classData) - 1] - classData["Open Seats"][0])
                    / len(days)])
# I didn't end up using the dataframes
ratesDF = pd.DataFrame(rates)
ratesDF.columns = ['Section', 'Rates']
className = ""
rates
    

[['001 (LEC)', 4.48],
 ['002 (LEC)', 4.64],
 ['010 (DIS)', 1.0],
 ['011 (DIS)', 1.6],
 ['012 (DIS)', 1.6],
 ['013 (DIS)', 1.6],
 ['014 (DIS)', 1.6],
 ['015 (DIS)', 1.6],
 ['016 (DIS)', 1.6],
 ['009 (DIS)', 0.0]]

In [14]:
# rate calculations
sections = data['Section'].unique()
sectionedRates = []
for className in sections:
    classData = data[data['Section']==className]
    classData = classData.sort_values(by=['Hour'], ascending=False).reset_index(drop=True)
    classData["Open Seats"][len(classData) - 1]
    if(len(classData) > 1) :
        sectionedRates.append([className,
                           (classData["Open Seats"][len(classData) - (int)(len(classData) / 2)] - classData["Open Seats"][0]) / len(days),
                           (classData["Open Seats"][len(classData) - 1] - classData["Open Seats"][len(classData) - (int)(len(classData) / 2)]) / len(days)])
#     for x in range(0,24):
#         classRates = rates.append((classData["Open Seats"][len(classData) - 1] - classData["Open Seats"][0])
#                     / len(days))
# I didn't end up using the dataframes
sectionedRatesDF = pd.DataFrame(sectionedRates)
sectionedRatesDF.columns = ['Section', 'Rate_1', "Rate_2"]
sectionedRatesDF

,Section,Rate_1,Rate_2
0,001 (LEC),1.64,2.84
1,002 (LEC),1.96,2.68
2,010 (DIS),1.00,0.00
3,011 (DIS),0.56,1.04
4,012 (DIS),0.80,0.80
5,013 (DIS),0.56,1.04
6,014 (DIS),0.56,1.04
7,015 (DIS),0.92,0.68
8,016 (DIS),0.68,0.92


In [10]:
# predicting the last day based on the number of open seats based on the rate
sections = data['Section'].unique()
finalDays = []
for className in sections:
    classData = data[data['Section']==className]
    classData = classData.sort_values(by=['Hour'], ascending=False).reset_index(drop=True)
    if (rates['Section' == className][1] > 0) :
        days_till_closing = classData["Open Seats"][len(classData) - 1] / (rates['Section' == className][1])
    finalDays.append([className, days[len(days) - 1 - math.floor(days_till_closing)]])
finalDaysDF = pd.DataFrame(finalDays)
finalDaysDF.columns = ['Section', 'Final_Day']
finalDaysDF
    
    
    

,Section,Final_Day
0,001 (LEC),2021-11-16
1,002 (LEC),2021-11-16
2,010 (DIS),2021-11-20
3,011 (DIS),2021-11-23
4,012 (DIS),2021-11-23
5,013 (DIS),2021-11-23
6,014 (DIS),2021-11-23
7,015 (DIS),2021-11-23
8,016 (DIS),2021-11-23
9,009 (DIS),2021-11-20
